# Postupci nelinearne optimizacije

In [7]:
from Matrica import Matrica, JedinicnaMatrica
from functools import reduce
import math
import numpy as np
import pandas as pd

In [8]:
# Ulazne velicine:
# - tocka: pocetna tocka pretrazivanja
# - h: pomak pretrazivanja
# - f: ciljna funkcija


def unimodalni(x0, func, h_value = 1.0):
    
    h = Matrica(1, 1, [[h_value]])
    tocka = x0.copy()
    left = tocka - h 
    right = tocka + h
    m = tocka
    step = 1
    
    fm = func.vrijednost(tocka)
    fl = func.vrijednost(left)
    fr = func.vrijednost(right)
    
    if fm < fr and fm < fl: 
        return left, right
    elif fm > fr:
        while True:
            left = m
            m = right
            fm = fr
            step *= 2
            right = tocka + (h * step)
            fr = func.vrijednost(right)
            if (fm <= fr): 
                break
    else: 
        while True:
            right = m
            m = left
            fm = fl
            step *= 2
            left = tocka - (h * step)
            fl = func.vrijednost(left)
            if fm <= fl: break
    return left, right

In [9]:
# Ulazne velicine:
# - a, b: pocetne granice unimodalnog intervala
# - e: preciznost

def zlatni_rez(a, b, func, epsilon = 1e-6, max_iter = 20000):
    
    k = 0.5 * (math.sqrt(5) - 1)
        
    c = b - k * (b - a)
    d = a + k * (b - a)
    fc = func.vrijednost(c)
    fd = func.vrijednost(d)
    
    br_iter = 0
    while((b[0][0] - a[0][0]) > epsilon and br_iter <= max_iter):
        if fc < fd:
            b = d 
            d = c
            c = b - k * (b - a)
            fd = fc
            fc = func.vrijednost(c)
        else:
            a = c
            c = d
            d = a + k * (b - a)
            fc = fd
            fd = func.vrijednost(d)
        br_iter += 1
    return (a + b) / 2

In [16]:
def koord_pretr(x0, func, epsilon = 1e-6, ispis = False):
    n = x0.br_stup
    xs = x0.copy()
    x = x0.copy()
    
    if ispis:
        print("n: ", n)
        print("x: ", x)
        
    flag = True
    while flag: 
        xs = x.copy()
        for i in range(n):
            if ispis: print("iter {0} -> x: {1}".format(i, x))
            xi = Matrica(1, 1, [[x0[0][i]]])
            e = Matrica(1, n)
            e[0][i] = 1.0
            x = minimum(xi, func1, e, ispis = ispis)
        for i in range(n):
            if not abs(xs[0][i] - x[0][i]) > epsilon: 
                flag = False
    return x

def minimum(x0, func, e, ispis = False):
    a, b = unimodalni(x0, func)
    if ispis: print("a: {0}, b: {1}".format(a, b))
    gamma_min = zlatni_rez(a, b, func)[0][0]
    result = e*gamma_min
    
    if ispis: 
        print("zlatni_rez: ", gamma_min)
        print("x0: {0}, e: {1}".format(x0, e))
        print("minimum {0} koordinate: {1}".format(i, result))
    return result
                

In [17]:
def koord_pretr2(x0, func, epsilon = 1e-6, max_iter = 200):
    n = x0.br_stup
    x = x0.copy()
    
    e = Matrica(1, n)
#     eps = Matrica(1, n, np.full((1, n), epsilon))
    
    br_iter = 0
    while True:
        br_iter +=1
        xs = x.copy()
        for i in range(n):
            xs_koordinata = Matrica(1, 1, [[xs[0][i]]])
            e[i] = 1
            a, b = unimodalni(xs_koordinata, func)
            l_min = zlatni_rez(a, b, func, epsilon)
#             print("l_min: ", l_min)
#             print("l_min*e: ", l_min*e)
#             print("x: ", x)
            x += l_min*e
#             print("new x: ", x)
            e[i] = 0
            for i in range(x.br_stup):
                if norma(x, xs) > epsilon: 
                    break
                return x
        if br_iter > max_iter:
            return x
        
def norma(x1, x2):
    suma = 0
    for i in range(x1.br_stup):
        suma += (x1[0][i] - x2[0][i])**2
    rezultat = suma**(1/2)
    return rezultat
        

# def pretraga(elementi):
#     l = elementi[0]
#     x[i][0] += l
#     rezultat = func.vrijednost(x)
#     x[i][0] -= l
#     return rezultat
    


In [18]:
def simplex(x0, func, pomak = 1, ispis = False, alfa = 1.0, beta = 0.5, gama = 2.0, sigma = 0.5, epsilon=1e-6):
    
    #tocke simplexa
    pocetna_tocka = x0.copy()
#     print("pocetna tocka simplexa: ", x0)
    X = [pocetna_tocka]
    for i in range(x0.br_stup):
        nova_tocka = x0.copy()
        nova_tocka[0][i] += pomak
        X.append(nova_tocka)

    #ispis simpleks tocaka
    if ispis: 
        print("X: ", end="\t")
        for i in range(len(X)):
            print(X[i], end=" ")
        print("\n")
    
    # - h, l, centroid -
    while True:
        h = index_max(X, func, ispis = ispis)
        l = index_min(X, func, ispis = ispis)
        xc = centroid(X, h)
        fc = func.vrijednost(xc)
            
        xr = refleksija(xc, X[h], alfa)
        fr = func.vrijednost(xr)
        
        #ispis rjesenja svakog pojednog koraka
        if ispis:
            print("h: ", h, " |l: ", l, " |xc: ", xc)
            print("fc: ", fc)
            print("xr, fr: ", xr, fr)
        
        fl = func.vrijednost(X[l])
#         print("fl: ", fl)
        if fr < fl:
            xe = ekspanzija(xr, xc, gama)
#             print("fe: ", fe)
            
            if func.vrijednost(xe) < fl:
                X[h] = xe.copy()
            else:
                X[h] = xr.copy()

        else:
            # provjera
            flag = True
            for j in range(len(X)): 
                if j != h:
                    if func.vrijednost(xr) < func.vrijednost(X[j]):
                        flag = False
                        break
            if flag:
                if func.vrijednost(xr) < func.vrijednost(X[h]):
                    X[h] = xr.copy()

                xk = kontrakcija(X[h], xc, beta)

                if func.vrijednost(xk) < func.vrijednost(X[h]):
                    X[h] = xk.copy()
                else:
                    pomak_prema_xl(X, l, sigma)
            else:
                X[h] = xr.copy()
                
        # Uvjet zaustavljanja
        s = 0
        for i in range(len(X)):
            s += (func.vrijednost(X[i]) - func.vrijednost(xc))**2
        if ispis:
            print("X[l], X[h], xc:")
            print(X[l], X[h], xc)
        
        if math.sqrt(s / float(len(X) - 1)) <= epsilon:
            return xc
            
        
# - MINIMUM I MAKSIMUM FJE CILJA        
def index_min(X, func, ispis = False):
    if ispis:
        print("ulaz u min")
        print("X: ", end = " ")
        for i in range(len(X)):
            print(X[i], end = " ")
        print("\n")
    i_min = 0
    for i in range(len(X)):
        fi = func.vrijednost(X[i])
        fi_min =  func.vrijednost(X[i_min])
        if ispis: print("{0} < {1}".format(fi, fi_min), end = " ")
        if fi < fi_min: 
            if ispis: print("DA")
            i_min = i
        else: 
            if ispis: print("NE")
#         if func.vrijednost(X[i]) < func.vrijednost(X[i_min]): i_min = i
    return i_min
 
def index_max(X, func, ispis = False):
    if ispis:
        print("ulaz u max")
        print("X: ", end=" ")
        for i in range(len(X)):
            print(X[i], end = " ")
        print("\n")
    i_max = 0
    for i in range(len(X)):
        fi = func.vrijednost(X[i])
        fi_max =  func.vrijednost(X[i_max])
        if ispis: print("{0} > {1}".format(fi, fi_max), end = " ")
        if fi > fi_max: 
            if ispis: print("DA")
            i_max = i
        else:
            if ispis: print("NE")
    return i_max

# - CENTROID -
def centroid(X, h):
#     print("X: ", X)
    tocke = X.copy()
#     print("X[h]: ", tocke[h])
    del tocke[h]
#     print("tocke: ", tocke)
    
    if len(tocke) == 1:
        centroid = tocke[0].elementi
#         print("1: ", centroid)
    else: 
        red = reduce(myadd, tocke)
        centroid = (1.0 / len(tocke)) * red
#         print("2: ", centroid)
    return Matrica(1, X[0].br_stup, [centroid[0]])

def myadd(a, b):
    return Matrica(a.br_red, a.br_stup, a.elementi + b.elementi)

# - REFLEKSIJA, EKSPANZIJA, KONTRAKCIJA -
def refleksija(xc, xh, alfa = 1.0):
    return ((1.0 + alfa) * xc) - (alfa * xh)

def ekspanzija(xc, xr, gama = 2.0):
    return ((1.0 - gama) * xc) + (gama * xr)

def kontrakcija(xh, xc, beta = 0.5):
    return ((1.0 - beta) * xc) + (beta * xh)

def pomak_prema_xl(X, l, sigma = 0.5):
    for i in range(len(X)):
        X[i] = sigma * (X[i] + X[l])
    return X

In [19]:
# x0 - početna točka
# xb - bazna točka
# xp - početna točka pretraživanja
# xn - točka dobivena pretraživanjem

def hooke_jeeves(x0, func, dx = 0.5, epsilon = 1e-6):

    xp = x0.copy()
    xb = x0.copy()
#     dx = Matrica(x0.br_red, x0.br_stup, np.full((x0.br_red, x0.br_stup), 0.5), True)
                 
    while True:
        xn = istrazi(func, xp, dx)
        
#         #ispis rjesenja svakog pojednog koraka
#         print("xb: ", xb.elementi[0][0], "xp: ", xp.elementi[0][0], "xn: ", xn.elementi[0][0], "dx: ", dx)
#         print("fxb: ", f_xb, "fxp: ", f_xp, "fxn: ", f_xn)
#         print("----------------------------------")
        
        f_xn = func.vrijednost(xn)
        f_xb = func.vrijednost(xb)
        f_xp = func.vrijednost(xp)
        
#         print(xb[0], '-' ,xp[0], '-',xn[0], "dx: ", dx)
#         print("{:4.2f} - {:4.2f} - {:4.2f}".format(f_xb, f_xp, f_xn))
#         print("----------------------------------")
        
        if f_xn < f_xb:
            xp = 2*xn - xb
            xb = xn.copy()
        else:
            dx = dx / 2
            xp = xb.copy()
        
        #uvjet zaustavljanja
        if dx < epsilon:
            return xb
        
        
def istrazi(func, xp, dx):
    x = xp.copy()
    for i in range(x.br_stup):
        p = func.vrijednost(x)
#         print("hj: ", x)
        x[0][i] += dx
#         print("hj: ", x, " += dx")
        n = func.vrijednost(x)
        if n > p:
            x[0][i] -= 2 * dx
#             print("hj: ", x, " -= 2 * dx")
            n = func.vrijednost(x)
            if n > p:
                x[0][i] += dx
#                 print("hj: ", x, " += dx")
#     print("hj: new_xn: ", x)
    return x

In [20]:
class Funkcija:
    
    def __init__(self, f):
        self.br_poziva = 0
        self.f = f
        self.vrijednosti = {}
    
    def reset(self):
        self.vrijednosti = {}
        sel1.br_poziva = 0
    
    def vrijednost(self, x):
        try:
#             print("self.vrijednosti: ", self.vrijednosti)
#             print("str x:", str(x))
            return self.vrijednosti[str(x)]
        except: 
            vrijednost = self.f(x)
            self.br_poziva += 1
            self.vrijednosti[str(x)] = vrijednost
            return vrijednost

def f1(vektor):
    x = vektor.elementi.flatten()
    result = 100 * (x[1] - (x[0])**2)**2 + (1 - x[0])**2
    return result

def f2(vektor):
    x = vektor.elementi.flatten()
    return (x[0] - 4)**2 + 4 * (x[1] - 2)**2

def f3(vektor):
    x = vektor.elementi.flatten()
    result = 0
    for i in range(len(x)):
        result += (x[i] - (i + 1))**2
    return result

def f4(vektor):
    x = vektor.elementi.flatten()
    return abs((x[0] - x[1]) * (x[0] + x[1])) + math.sqrt(x[0]**2 + x[1]**2)

def f6(v):
    x = v.elementi.flatten()
    suma = 0
    try:
        for i in range(len(x)):
            suma += x[i]**2
    except TypeError: 
        suma = v**2
    sinus = math.sin(math.sqrt(suma))
    razlomak = ((sinus**2) - 0.5) / ((1 + 0.001 * suma)**2)
    return 0.5 + razlomak

## Zadaci

1. Definirajte jednodimenzijsku funkciju br. 3, koja će imati minimum u točki 3. Kao početnu točku
pretraživanja postavite točku 10. Primijenite sve postupke na rješavanje ove funkcije te ispišite
pronađeni minimum i broj evaluacija funkcije za svaki pojedini postupak. Probajte sve više
udaljavati početnu točku od minimuma i probajte ponovo pokrenuti navedene postupke. Što možete
zaključiti? 

In [21]:
#pronadeni minimum i broj evaluacija za svaki pojedini postupak
#sto vise se udaljavati od tocke te ponovno pokretati postupak
for i in range(10, 500, 50):
    x0 = Matrica(1, 1, [[i]], True)

    func_hj = Funkcija(f3)
    func_sim = Funkcija(f3)
    func_zl = Funkcija(f3)
    func_koord = Funkcija(f3)
    

    a, b = unimodalni(x0, func_sim)
    
    print("pocetna tocka je: " + str(i))
    hj = hooke_jeeves(x0, func_hj)
    print("hj", func_hj.br_poziva, hj)

    sim = simplex(x0, func_sim)
    print("si", func_sim.br_poziva, sim)

    zl = zlatni_rez(a, b, func_zl)
    print("zl", func_zl.br_poziva, zl)

    koord = koord_pretr(x0, func_koord)
    print("ko", func_koord.br_poziva, koord)
    print("-----------------------------")
#     break

pocetna tocka je: 10
hj 56 [[1.]]
si 32 [[1.]]
zl 36 [[1.00000014]]


TypeError: 'set' object is not callable

2. Primijenite simpleks po Nelderu i Meadu, Hooke-Jeeves postupak te pretraživanje po koordinatnim
osima na funkcije 1 - 4 uz zadane parametre i početne točke (broj varijabli funkcije 3 najmanje 5).
Za svaki postupak i svaku funkciju odredite minimum koji su postupci pronašli i potrebni broj
evaluacija funkcije cilja koji je potreban do konvergencije (prikažite tablično). Što možete zaključiti
iz rezultata? 

In [26]:
pocetne_tocke = [[-1.9, 2], [0.1, 0.3], [0, 0, 0, 0, 0], [5.1, 1.1]]
minimumi = [[1, 1], [4, 2], [1, 2, 3, 4, 5], [0, 0]]
f = [f1, f2, f3, f4]

for i in range(4):
    
    print("f{0} \t pocetna tocka: {1} \t minimum: {2}".format(i, pocetne_tocke[i], minimumi[i]))
    x0 = Matrica(1, len(pocetne_tocke[i]), [pocetne_tocke[i]])

#     func_hj = Funkcija(f[i])
#     func_sim = Funkcija(f[i])
    func_koord = Funkcija(f[i])

#     hj = hooke_jeeves(x0, func_hj)
#     print("hj", func_hj.br_poziva, hj)

#     sim = simplex(x0, func_sim)
#     print("si", func_sim.br_poziva, sim)

    koord = koord_pretr(x0, func_koord)
    print("ko", func_koord.br_poziva, koord)
    print("-------------------------------------------------------------------")

f0 	 pocetna tocka: [-1.9, 2] 	 minimum: [1, 1]


IndexError: index 1 is out of bounds for axis 0 with size 1

3. Primijenite postupak Hooke-Jeeves i simpleks po Nelderu i Meadu na funkciju 4 uz početnu točku
(5, 5). Objasnite rezultate! 

In [13]:
x0 = Matrica(1, 2, [[5, 5]])
# print(x0)
func_hj = Funkcija(f4)
func_sim = Funkcija(f4)

print(hooke_jeeves(x0, func_hj))
print(simplex(x0, func_sim))

[[5. 5.]]
[[3.19264902e-08 7.81653725e-07]]


4. Primijenite simpleks po Nelderu i Meadu na funkciju 1. Kao početnu točku postavite točku (0.5,0.5).
Provedite postupak s nekoliko različitih koraka za generiranje početnog simpleksa (primjerice iz
intervala od 1 do 20) i zabilježite potreban broj evaluacija funkcije cilja i pronađene točke
minimuma. Potom probajte kao početnu točku postaviti točku (20,20) i ponovo provesti eksperiment.
Što možete zaključiti?

In [36]:
x0 = Matrica(1, 2, [[0.5, 0.5]])
x1 = Matrica(1, 2, [[20, 20]])

for pomak in range(1, 21):
    
    func_sim1 = Funkcija(f1)
    func_sim2 = Funkcija(f1)
    
    y1 = simplex(x0, func_sim1, pomak, ispis = False)
    y2 = simplex(x1, func_sim2, pomak, ispis = False)
    
    print("pomak:", pomak, "\t", func_sim1.br_poziva, '{:02.6f}'.format(y1[0][0]), '{:02.6f}'.format(y1[0][1]), "\t", func_sim2.br_poziva, '{:02.6f}'.format(y2[0][0]), '{:02.6f}'.format(y1[0][1]))
    break

pomak: 1 	 88 0.999908 0.999777 	 13441 1.002700 0.999777


5. Primijenite jedan postupak optimizacije na funkciju 6 u dvije dimenzije, tako da postupak pokrećete
više puta iz slučajno odabrane početne točke u intervalu [-50,50]. Možete li odrediti vjerojatnost
pronalaženja globalnog optimuma na ovaj način? (smatramo da je algoritam locirao globalni
minimum ako je nađena vrijednost funkcije cilja manja od 4
10−
). 

In [ ]:
from random import randint

iteration = 0
for i in range(50):
    while True:
        iteration += 1
        x1 = randint(-50, 50)
        x2 = randint(-50, 50)
        tocka = Matrica(1, 2, [[x1, x2]])

        func_sim = Funkcija(f6)
        rez = simplex(tocka, func_sim)

        if f6(rez) < 1e-4: break
(50/iteration) * 50

### Provjere

In [ ]:
# Provjera mnozenja matrica

v1 = Matrica(1, 3, [[1, 2, 3]])
v2 = Matrica(3, 1, [[1], [2], [3]])
m1 = Matrica(3, 3, [[1, 2, 3], [1, 1, 1], [1, 2, 3]])

print("m1/3")
print(m1/3)

print("1/3 * (m1)")
print(1/3 * m1)

# print(m1*v1)
print("m1*v2")
print(m1*v2)
print("v1*m1")
print(v1*m1)
print("m1*m1")
print(m1*m1)
print("v1*v2")
print(v1*v2)
print("v2*v1")
print(v2*v1)

In [ ]:
# pomak prema X[l]
sim = [Matrica(1, 2, [[2, 3]]), Matrica(1, 2, [[1, 1]]), Matrica(1, 2, [[3, 4]]), Matrica(1, 2, [[4, 5]]), Matrica(1, 2, [[5, 6]])]
novi = pomak_prema_xl(sim, 1)
for m in novi:
    print(m)
    print(m[0][0])

In [ ]:
# provjera nove verzije fje __init__ za matrice 

# three = np.array(3)
# four = np.array(4)
# five = np.array(5)
three = [3]
four = [4]
five = [5]
np.array([three, four, five])
m1 = Matrica(3, 1, [[4, 5, 6]])
m2 = Matrica(1, 3, [[4, 5, 6]])
print(m1[0][0])
print(m2[0][0])

In [ ]:
# provjera break-a
gh = 0
while gh < 3:
    gh += 1
    for fj in range(4):
        if gh > 2: break
        print("inside for: gh=" + str(gh))
    print("outside for: gh=" + str(gh))

In [ ]:
print(Matrica(1, 10, [np.zeros(10)]))
print(Matrica(1, 10))

In [16]:
polje = np.array([6.5000000e+00, 7.3850465e+06, 2.6406500e+04])

In [ ]:
print(polje)

In [ ]:
polje[0]<polje[2]

In [ ]:
polje[0]<polje[1]

In [ ]:
np.argmin(polje)

In [19]:
lista = [2, 3, 4, 5]
if not lista:
    print("tocno")

In [20]:
[x for x in range(len(lista))]

[0, 1, 2, 3]

In [22]:
def simplex2(x0, func, pomak = 1, ispis = False, alfa = 1.0, beta = 0.5, gama = 2.0, sigma = 0.5, epsilon=1e-6):
    
    #tocke simplexa
    pocetna_tocka = x0.copy()
#     print("pocetna tocka simplexa: ", x0)
    X = [pocetna_tocka]
    for i in range(x0.br_stup):
        nova_tocka = x0.copy()
        nova_tocka[0][i] += pomak
        X.append(nova_tocka)
    
    cache = [func.vrijednost(X[i]) for i in range(len(X))]

    #ispis simpleks tocaka
    if ispis: 
        print("X: ", end="\t")
        for i in range(len(X)):
            print(X[i], end=" ")
        print("\n")
        
        print("fx: ", end="\t")
        print(cache)
    
    # - h, l, centroid -
    while True:
        h, l = index_min_i_max_fje(X, cache)
        xc = centroid(X, h)
        fc = func.vrijednost(xc)
            
        xr = refleksija(xc, X[h], alfa)
        fr = func.vrijednost(xr)
        
        #ispis rjesenja svakog pojednog koraka
        if ispis:
            print("h: ", h, " |l: ", l, " |xc: ", xc)
#             print("fc: ", fc)
#             print("xr, fr: ", xr, fr)
        
        if fr < cache[l]:
            xe = ekspanzija(xr, xc, gama)
            fe = func.vrijednost(xe)
#             print("fe: ", fe)
            
            if fe < cache[l]:
                X[h] = xe.copy()
                cache[h] = fe
            else:
                X[h] = xr.copy()
                cache[h] = fr
#             func.vrijednosti.pop(str(X[h]))

        else:
            # provjera
            flag = True
            for j in range(len(X)): 
                if j != h:
                    if fr < cache[j]:
                        flag = False
                        break
            if flag:
                if fr < cache[h]:
                    X[h] = xr.copy()
                    cache[h] = fr
#                     func.vrijednosti.pop(str(X[h]))

                xk = kontrakcija(X[h], xc, beta)
                fk = func.vrijednost(xk)

                if fk < cache[h]:
                    X[h] = xk.copy()
                    cache[h] = fk
#                     func.vrijednosti.pop(str(X[h]))
                else:
                    pomak_prema_xl(X, l, sigma)
                    cache = [func.vrijednost(X[i]) for i in range(len(X))]
            else:
                X[h] = xr.copy()
                cache[h] = fr
#                 func.vrijednosti.pop(str(X[h]))
                
        # Uvjet zaustavljanja
        s = 0
        for i in range(len(X)):
            s += (func.vrijednost(X[i]) - func.vrijednost(xc))**2
#         if ispis:
#             print("X[l], X[h], xc:")
#             print(X[l], X[h], xc)
        
        if math.sqrt(s / float(len(X) - 1)) <= epsilon:
            return xc
            
        
# - MINIMUM I MAKSIMUM FJE CILJA        

def index_min_i_max_fje(X, cache, ispis = False):
    i_min = 0
    i_max = 0
    fi_min = cache[0]
    fi_max = cache[0]
#     fi_min = func.vrijednost(X[i_min])
#     fi_max = func.vrijednost(X[i_max])
    
    for i in range(1, len(X)):
#         fi = func.vrijednost(X[i])
        fi = cache[i]
        if fi > fi_max:
            i_max = i
            fi_max = fi
        elif fi < fi_min:
            i_min = i
            fi_min = fi
            
    return i_max, i_min


# def index_min(X, func, ispis = False):
#     if ispis:
#         print("ulaz u min")
#         print("X: ", end = " ")
#         for i in range(len(X)):
#             print(X[i], end = " ")
#         print("\n")
#     i_min = 0
#     for i in range(len(X)):
#         fi = func.vrijednost(X[i])
#         fi_min =  func.vrijednost(X[i_min])
#         if ispis: print("{0} < {1}".format(fi, fi_min), end = " ")
#         if fi < fi_min: 
#             if ispis: print("DA")
#             i_min = i
#         else: 
#             if ispis: print("NE")
# #         if func.vrijednost(X[i]) < func.vrijednost(X[i_min]): i_min = i
#     return i_min
 
# def index_max(X, func, ispis = False):
#     if ispis:
#         print("ulaz u max")
#         print("X: ", end=" ")
#         for i in range(len(X)):
#             print(X[i], end = " ")
#         print("\n")
#     i_max = 0
#     for i in range(len(X)):
#         fi = func.vrijednost(X[i])
#         fi_max =  func.vrijednost(X[i_max])
#         if ispis: print("{0} > {1}".format(fi, fi_max), end = " ")
#         if fi > fi_max: 
#             if ispis: print("DA")
#             i_max = i
#         else:
#             if ispis: print("NE")
#     return i_max

# - CENTROID -
def centroid(X, h):
#     print("X: ", X)
    tocke = X.copy()
#     print("X[h]: ", tocke[h])
    del tocke[h]
#     print("tocke: ", tocke)
    
    if len(tocke) == 1:
        centroid = tocke[0].elementi
#         print("1: ", centroid)
    else: 
        red = reduce(myadd, tocke)
        centroid = (1.0 / len(tocke)) * red
#         print("2: ", centroid)
    return Matrica(1, X[0].br_stup, [centroid[0]])

def myadd(a, b):
    return Matrica(a.br_red, a.br_stup, a.elementi + b.elementi)

# - REFLEKSIJA, EKSPANZIJA, KONTRAKCIJA -
def refleksija(xc, xh, alfa = 1.0):
    return ((1.0 + alfa) * xc) - (alfa * xh)

def ekspanzija(xc, xr, gama = 2.0):
    return ((1.0 - gama) * xc) + (gama * xr)

def kontrakcija(xh, xc, beta = 0.5):
    return ((1.0 - beta) * xc) + (beta * xh)

def pomak_prema_xl(X, l, sigma = 0.5):
    for i in range(len(X)):
        X[i] = sigma * (X[i] + X[l])
    return X